In [ ]:
from src.Layer import Layer
from src.WFSLayer import WFSLayer
from src.Geometry import Geometry
from src.OSMLayer import OSMLayer
from src.Isochrone import Isochrone
import src.constants as c
import geopandas as gpd

import os
import dotenv
import copy

dotenv.load_dotenv(override=True)

postgres_url = os.getenv('POSTGRES_URL')

plangebied = Geometry(geom_wkt=c.ZUID_HOLLAND_GEOM).add_buffer(buffer_distance=50000)
plangebied_4326 = copy.copy(plangebied).reproject()

In [ ]:
# Get home location data
gpkg_path = './data/the_parents.gpkg'
the_parents = Layer(postgres_url=postgres_url)
the_parents.from_geopackage(path=gpkg_path)

the_parents.to_postgres(table_name='the_parents', schema_name='geonest')

# Get isochrones for home locations
the_parents_isochrones = Isochrone(
    postgres_url=postgres_url,
    contours_minutes=[30, 45],
    api_key=os.getenv('MAPBOX_API_KEY'),
    points=the_parents.reproject('EPSG:4326').gdf
)

the_parents_isochrones.reproject('EPSG:28992').to_postgres(table_name='the_parents_isochrones', schema_name='geonest')
the_parents_isochrones.to_postgres(table_name='the_parents_isochrones', schema_name='geonest')


In [ ]:
# Get all climbing gyms in the area
layer = OSMLayer(
    postgres_url=postgres_url,
    tags={'sport': 'climbing'},
    area=plangebied
)
layer.to_postgres(table_name='osm_climbing', schema_name='geonest')

# Get isochrones for climbing gyms
isochrones = Isochrone(
    postgres_url=postgres_url,
    contours_minutes=[15, 30],
    api_key=os.getenv("MAPBOX_API_KEY"),
    points=layer.gdf
)
isochrones.reproject('EPSG:28992').to_postgres(table_name='osm_climbing_isochrone', schema_name='geonest')


In [ ]:
# Get buurten from CBS
CBS_WIJKENENBUURTEN_WFS_PARAMS = {
    "service": "WFS",
    "version": "2.0.0",
    "request": "GetFeature",
    "typeName": "wijkenbuurten:buurten",
    "outputFormat": "application/json",
    "count": 1000,
    "startindex": 0
}

layer = WFSLayer(
    postgres_url=postgres_url,
    wfs_url='https://service.pdok.nl/cbs/wijkenbuurten/2023/wfs/v1_0',
    params=CBS_WIJKENENBUURTEN_WFS_PARAMS,
    area=plangebied,
)
layer.to_postgres(table_name='buurten', schema_name='geonest')


In [ ]:
from shapely.geometry import Point
import numpy as np

# Get all beaches in the area
beaches_layer = OSMLayer(
    postgres_url=postgres_url,
    tags={'natural': 'beach'},
    area=plangebied_4326
)

# Get all seas in the area
seas_layer = OSMLayer(
    postgres_url=postgres_url,
    tags={'place': 'sea'},
    area=plangebied_4326
)

sea_beaches_layer = Layer(
    postgres_url=postgres_url,
    gdf=beaches_layer.gdf[beaches_layer.gdf.geometry.intersects(seas_layer.gdf.unary_union)]
)
sea_beaches_layer.to_postgres(table_name='osm_sea_beaches', schema_name='geonest')

# Get isochrones for beaches
# isochrones = Isochrone(
#     postgres_url=postgres_url,
#     contours_minutes=[15, 30],
#     api_key=os.getenv("MAPBOX_API_KEY"),
#     points=sea_beaches_layer.gdf
# )
# isochrones.reproject('EPSG:28992').to_postgres(table_name='osm_sea_beaches_isochrone', schema_name='geonest')

# Get isochrones for beaches
isochrones = Isochrone(
    postgres_url=postgres_url,
    contours_minutes=[30],
    api_key=os.getenv("MAPBOX_API_KEY"),
    points=sea_beaches_layer.gdf,
    mode='cycling'
)
isochrones.reproject('EPSG:28992').to_postgres(table_name='osm_sea_beaches_isochrone_cycling', schema_name='geonest')
